![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.columns

In [ ]:
# Переименуем столбец Cuisine Style для удобства
data.columns = (['Restaurant_id', 'City', 'Cuisine_Style', 'Ranking', 'Price Range', 
                 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA', 'sample', 'Rating'])

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.
Вынесем функции в предобработку.

In [ ]:
def fix_cuisine_style(x):        # функция для преобразования колонки 'cuisine_style'
    if type(x) != float:
        if x == None:
            return None
        x = x.replace(' ', '')
        x = x.replace('[', '')
        x = x.replace(']', '')
        return x

In [ ]:
def fix_reviews(x):     # функция для преобразования колонки 'reviews'
    if pd.isnull(x):
        return x
    if x == 'nan':
        return None
    x = x.replace('[[', '')
    x = x.replace(']]', '')
    x = x.replace('[', '')
    x = x.replace(']', '')
    return x

In [ ]:
def rew(x):      # функция замены символа по указанному шаблону на 'None'
    if pd.isnull(x):
        return x
    if x == 'nan':
        return None
    rew = re.sub(r'^\S\s$', 'None', str(x))
    return rew

In [ ]:
def IQR_col_num(column):                     # Функция для числовых столбцов для определения 
    Q1 = data[column].quantile(0.25)         # величины межквартильного размаха и границ выбросов
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    print('25-й перцентиль: {}, '.format(Q1), '75-й перцентиль: {}, '.format(Q3),
         'IQR: {}, '.format(IQR), 'Границы выбросов: [{f}, {l}].'.format(f = Q1 - 1.5*IQR, l = Q3 + 1.5*IQR))


def describe_col_IQR_l(column):              # функция, возвращающая значение левой границы выброса
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    l = Q1 - 1.5*IQR
    return l


def describe_col_IQR_r(column):              # функция, возвращающая значение правой границы выброса
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    r = Q3 + 1.5*IQR
    return r

In [ ]:
def rank_range(x):                                                          # функция, группирующая значения столбца 'ranking'
    if data['Ranking'].min() <= x < data['Ranking'].quantile(0.25):         # по заданному условию и возвращающая значения [1-5]
        return 1
    elif data['Ranking'].quantile(0.25) <= x < data['Ranking'].quantile(0.50):
        return 2
    elif data['Ranking'].quantile(0.50) <= x < data['Ranking'].quantile(0.75):
        return 3
    elif data['Ranking'].quantile(0.75) <= x <= describe_col_IQR_r('Ranking'):
        return 4
    elif x > describe_col_IQR_r('Ranking'):   # x > (Q3 + 1.5*IQR)
        return 5
    return None

In [ ]:
def numb_review_range(x):                                                                    # функция, группирующая значения столбца 
    if data['Number of Reviews'].min() <= x < data['Number of Reviews'].quantile(0.25):      # 'numb_of_reviews' и возвращающая значения [1-5]
        return 1
    elif data['Number of Reviews'].quantile(0.25) <= x < data['Number of Reviews'].quantile(0.50):
        return 2
    elif data['Number of Reviews'].quantile(0.50) <= x < data['Number of Reviews'].quantile(0.75):
        return 3
    elif data['Number of Reviews'].quantile(0.75) <= x <= describe_col_IQR_r('Number of Reviews'):
        return 4
    elif x > describe_col_IQR_r('Number of Reviews'):   # x > (Q3 + 1.5*IQR)
        return 5
    return None

In [ ]:
def cuis_style(row):   # функция для преобразования колонки 'cuisine_style' (поиск по столбцу 'url_ta')
    for word in ['Italian', 'Spanish', 'Portuguese', 'Greek', 'Turkish', 'Moroccan', 'Mediterranean']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Mediterranean'"
            return row['Cuisine_Style']
    for word in ['Japanese', 'Chinese', 'Thai', 'Vietnamese', 'Korean', 'Indonesian', 'Asian', 'Filipino']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Asian'"
            return row['Cuisine_Style']
    for word in ['Scandinavian']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Scandinavian'"
            return row['Cuisine_Style']
    for word in ['Indian']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Indian'"
            return row['Cuisine_Style']    
    for word in ['French']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'European'"
            return row['Cuisine_Style']    
    for word in ['Mexican']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Mexican'"
            return row['Cuisine_Style']
    return None

In [ ]:
def cuis(row):   # функция для преобразования колонки 'cuisine_style' (поиск по столбцу 'url_ta')
    for word in ['Ital', 'Spanis', 'Greek', 'Turkish', 'Moroc', 'Mediter', 'Neap', 'Milan', 'pasta',
                'Rome', 'Madrid', 'tapas', 'catalan', 'ravioli']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Mediterranean'"
            return row['Cuisine_Style']
    for word in ['Japan', 'Sush', 'Chines', 'Thai', 'Vietna', 'Korea', 'Indones', 'Malays', 'Tibet', 'Taiwan', 'Asia',
                'ramen']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Asian'"
            return row['Cuisine_Style']
    for word in ['Austri', 'Hungari', 'Germ', 'Polish', 'Sloven', 'Swiss', 'Czech', 'Belgi', 'Croat',
                'Amsterd', 'bavarian', 'Hamburg', 'Zurich']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'CentralEuropean'"
            return row['Cuisine_Style']    
    for word in ['Hala', 'Leban', 'Isra', 'Pers', 'Arab', 'Kosher', 'MiddleEast', 'MidEast', 'falafel']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'MiddleEastern'"
            return row['Cuisine_Style']   
    for word in ['Latin', 'Argent', 'Brazil', 'Peru', 'Venez', 'SouthAmer']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'SouthAmerican'"
            return row['Cuisine_Style']    
    for word in ['Dani', 'Swed', 'Norw', 'Scand']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Scandinavian'"
            return row['Cuisine_Style']
    for word in ['Pakist', 'Nepal', 'Banglad', 'SriLan', 'Indi', 'masala']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Indian'"
            return row['Cuisine_Style']    
    for word in ['Delicatessen', 'French', 'Croissan', 'BAGUETT']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'European'"
            return row['Cuisine_Style']    
    for word in ['Carib', 'Jamai', 'Cuba', 'CentralAmer']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'CentralAmerican'"
            return row['Cuisine_Style']
    for word in ['Barbec', 'Gril', 'BBQ']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'American'"
            return row['Cuisine_Style']
    for word in ['Ethiop', 'Tunis', 'Afri', 'Nigerian']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'African'"
            return row['Cuisine_Style']    
    for word in ['Soup', 'Balt', 'Russ', 'Afghan', 'EasternEu']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'EasternEuropean'"
            return row['Cuisine_Style']   
    for word in ['Brit', 'Dutch', 'Irish', 'Scotti', 'Dublin', 'Portug', 'Lisbon']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'NorthWesternEuropean'"
            return row['Cuisine_Style']
    for word in ['Contemp', 'Fusi']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Fusion'"
            return row['Cuisine_Style']   
    for word in ['Steakh', 'Cafe', 'Caffe', 'buffet', 'gelato']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Cafe'"
            return row['Cuisine_Style']   
    for word in ['Pizz', 'StreetFo', 'FastF', 'Pitstop', 'burger', 'fries', 'bistro', 'kebab', 'sandwich',
                'street foo', 'KFC', 'donalds', 'fast foo']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'FastFood'"
            return row['Cuisine_Style']
    for word in ['Pub', 'WineBa', 'Gastropu', 'BrewPu', 'Bar', 'cocktail', 'football', 'beer']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'Bar'"
            return row['Cuisine_Style']
    for word in ['Veg', 'Health', 'GlutenFre', 'Gluten Fre']:
        if word.lower() in row.URL_TA.lower():
            row['Cuisine_Style'] = "'VegetarianFriendly'"
            return row['Cuisine_Style']
    return None

In [ ]:
def f_cuis_style(row):   # функция для преобразования колонки 'cuisine_style' (поиск по столбцу 'reviews')
    for word in ['Italian', 'Spanish', 'Portuguese', 'Greek', 'Turkish', 'Moroccan', 'Mediterranean']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Mediterranean'"
            return row['Cuisine_Style']
    for word in ['Japanese', 'Chinese', 'Thai', 'Vietnamese', 'Korean', 'Indonesian', 'Asian', 'Filipino']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Asian'"
            return row['Cuisine_Style']
    for word in ['Scandinavian']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Scandinavian'"
            return row['Cuisine_Style']
    for word in ['Indian']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Indian'"
            return row['Cuisine_Style']    
    for word in ['French']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'European'"
            return row['Cuisine_Style']    
    for word in ['Mexican']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Mexican'"
            return row['Cuisine_Style']
    return None

In [ ]:
def f_cuis(row):   # функция для преобразования колонки 'cuisine_style' (поиск по столбцу 'reviews')
    for word in ['Ital', 'Spanis', 'Greek', 'Turkish', 'Moroc', 'Mediter', 'Neap', 'Milan', 'pasta',
                'Rome', 'Madrid', 'tapas', 'catalan', 'ravioli']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Mediterranean'"
            return row['Cuisine_Style']
    for word in ['Japan', 'Sush', 'Chines', 'Thai', 'Vietna', 'Korea', 'Indones', 'Malays', 'Tibet', 'Taiwan', 'Asia',
                'ramen']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Asian'"
            return row['Cuisine_Style']
    for word in ['Austri', 'Hungari', 'Germ', 'Polish', 'Sloven', 'Swiss', 'Czech', 'Belgi', 'Croat',
                'Amsterd', 'bavarian', 'Hamburg', 'Zurich']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'CentralEuropean'"
            return row['Cuisine_Style']    
    for word in ['Hala', 'Leban', 'Isra', 'Pers', 'Arab', 'Kosher', 'MiddleEast', 'MidEast', 'falafel']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'MiddleEastern'"
            return row['Cuisine_Style']   
    for word in ['Latin', 'Argent', 'Brazil', 'Peru', 'Venez', 'SouthAmer']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'SouthAmerican'"
            return row['Cuisine_Style']    
    for word in ['Dani', 'Swed', 'Norw', 'Scand']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Scandinavian'"
            return row['Cuisine_Style']
    for word in ['Pakist', 'Nepal', 'Banglad', 'SriLan', 'Indi', 'masala']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Indian'"
            return row['Cuisine_Style']    
    for word in ['Delicatessen', 'French', 'Croissan', 'BAGUETT']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'European'"
            return row['Cuisine_Style']    
    for word in ['Carib', 'Jamai', 'Cuba', 'CentralAmer']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'CentralAmerican'"
            return row['Cuisine_Style']
    for word in ['Barbec', 'Gril', 'BBQ']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'American'"
            return row['Cuisine_Style']
    for word in ['Ethiop', 'Tunis', 'Afri', 'Nigerian']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'African'"
            return row['Cuisine_Style']    
    for word in ['Soup', 'Balt', 'Russ', 'Afghan', 'EasternEu']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'EasternEuropean'"
            return row['Cuisine_Style']   
    for word in ['Brit', 'Dutch', 'Irish', 'Scotti', 'Dublin', 'Portug', 'Lisbon']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'NorthWesternEuropean'"
            return row['Cuisine_Style']
    for word in ['Contemp', 'Fusi']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Fusion'"
            return row['Cuisine_Style']   
    for word in ['Steakh', 'Cafe', 'Caffe', 'buffet', 'gelato']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Cafe'"
            return row['Cuisine_Style']   
    for word in ['Pizz', 'StreetFo', 'FastF', 'Pitstop', 'burger', 'fries', 'bistro', 'kebab', 'sandwich',
                'street foo', 'KFC', 'donalds', 'fast foo']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'FastFood'"
            return row['Cuisine_Style']
    for word in ['Pub', 'WineBa', 'Gastropu', 'BrewPu', 'Bar', 'cocktail', 'football', 'beer']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'Bar'"
            return row['Cuisine_Style']
    for word in ['Veg', 'Health', 'GlutenFre', 'Gluten Fre']:
        if word.lower() in row.Reviews.lower():
            row['Cuisine_Style'] = "'VegetarianFriendly'"
            return row['Cuisine_Style']
    return None

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Возьмём столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
data['Number_of_Reviews_isNAN']

In [ ]:
data['Number of Reviews'].nunique()

In [ ]:
display(data['Number of Reviews'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data['Number of Reviews'].value_counts()>10).sum())
data.loc[:, ['Number of Reviews']].info()

**** 
Итого у нас 1573 уникальных значений в колонке с количеством отзывов и всего 50000 строк. Столбец числовой и содержит пропуски - 3200 строк.

In [ ]:
data[['Reviews', 'Number of Reviews']][data['Number of Reviews'].isna() == True]

Произведем предобработку столбца 'Reviews', применим функции

In [ ]:
data['Reviews'].isnull().value_counts()

In [ ]:
data['Reviews'] = data['Reviews'].apply(fix_reviews)

In [ ]:
data['Reviews'] = data['Reviews'].apply(rew)

In [ ]:
data[['Reviews', 'Number of Reviews']][data['Number of Reviews'].isna() == True]

In [ ]:
data[['Reviews']][(data['Number of Reviews'].isna() == True) & 
                  (data['Reviews'] == 'None')].count()

In [ ]:
# Заменим пропуски на 0 там, где в колонке 'Reviews' нет отзывов 
# а в остальных случаях заменим пропуски на значение медианы:
def fillna_numb_rev(row):  
    if row.Reviews == 'None':
        return 0
    else:
        return data['Number of Reviews'].quantile(0.50)

In [ ]:
data['Number of Reviews'][data['Number of Reviews'].isna() == True] = data.apply(lambda row: fillna_numb_rev(row), axis=1)

In [ ]:
display(data['Number of Reviews'].value_counts())
data.loc[:, ['Number of Reviews']].info()

Как видим, все пропуски заполнены.

In [ ]:
# Возьмём столбец Price Range
data['Price_Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')

In [ ]:
display(data['Price Range'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data['Price Range'].value_counts()>10).sum())
data.loc[:, ['Price Range']].info()

In [ ]:
data['Price Range'].isnull().value_counts()

In [ ]:
data['Price Range'].describe()

Столбец строковый и содержит пропуски - 17361 строка. Можно заметить, что 23041 ресторан относится к среднему ценовому сегменту и встречается чаще всего.

In [ ]:
# Заменим пропуски на значение среднего ценового сегмента
data['Price Range'] = data['Price Range'].fillna(data['Price Range'].describe().top)

In [ ]:
display(data['Price Range'].value_counts())
data.loc[:, ['Price Range']].info()

Все пропуски заполнены.

In [ ]:
# Изучим столбец Reviews
data['Reviews_isNAN'] = pd.isna(data['Reviews']).astype('uint8')

In [ ]:
display(data['Reviews'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data['Reviews'].value_counts()>10).sum())
data.loc[:, ['Reviews']].info()

Итого у нас 41857 уникальных значений в колонке с отзывами и всего 50000 строк. Столбец строковый и содержит пропуски - 2 строки. Также содержит None в количестве 8112.

In [ ]:
data['Reviews'].describe()

In [ ]:
data[['Reviews', 'Number of Reviews']][(data['Reviews'].isna() == True) |
                                       (data['Reviews'] == 'None')]

In [ ]:
# Заменим пропуски и None на 'zero' там, где в колонке 'Number of Reviews' нет отзывов ('Number of Reviews' == 0) 
# а в остальных случаях заменим пропуски на значение 'not good, not bad':
def fillna_rev(row):  
    if row['Number of Reviews'] == 0:
        return 'zero'
    else:
        return 'not good, not bad'

In [ ]:
data['Reviews'][data['Reviews'].isna() == True] = data.apply(lambda row: fillna_rev(row), axis=1)

In [ ]:
data['Reviews'][data['Reviews'] == 'None'] = data.apply(lambda row: fillna_rev(row), axis=1)

In [ ]:
display(data['Reviews'].value_counts())
data.loc[:, ['Reviews']].info()

In [ ]:
data[['Reviews', 'Number of Reviews']][data['Reviews'] == 'not good, not bad']

In [ ]:
data[['Reviews', 'Number of Reviews']][data['Reviews'] == 'zero']

Все пропуски заполнены.

In [ ]:
# Изучим столбец Cuisine Style
data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine_Style']).astype('uint8')

In [ ]:
display(data['Cuisine_Style'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data['Cuisine_Style'].value_counts()>10).sum())
data.loc[:, ['Cuisine_Style']].info()

In [ ]:
data.Cuisine_Style.isna().sum()

Итого у нас 10731 уникальных значений и всего 50000 строк. Столбец строковый, содержит пропуски - 11590 строк. Заполним их после обработки столбца 'Cuisine_Style'

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.info()

In [ ]:
data.nunique(dropna=False)

#### Возьмем признак "City"

In [ ]:
display(data['City'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data['City'].value_counts()>10).sum())
data.loc[:, ['City']].info()

In [ ]:
data.City.nunique()

Столбец строковый. Итого у нас 31 уникальное значение в колонке с названиями городов и всего 50000 строк. Все города встречаются больше 10 раз. Пустые значения отсутствуют.

In [ ]:
# Сократим количество уникальных значений в признаке
cities_with_freqs = list(data.City.value_counts())

In [ ]:
top_cities_count = int(np.percentile(cities_with_freqs, 25))
top_cities_count

In [ ]:
cities_to_throw_away = list(data.City.value_counts()[data.City.value_counts() < top_cities_count].index)
cities_to_throw_away

In [ ]:
data.loc[data['City'].isin(cities_to_throw_away), 'City'] = 'other_city'

In [ ]:
data['City'].value_counts()

Количество уникальных значений в признаке city сократилось до:

In [ ]:
data.City.nunique()

#### Вернёмся к колонке 'price range' - диапазон цен в ресторане

In [ ]:
display(data['Price Range'].value_counts())
data.loc[:, ['Price Range']].info()

Столбец строковый, без пропусков

#### Возьмем признак "Ranking" - место, которое занимает данный ресторан среди всех ресторанов своего города

In [ ]:
display(data['Ranking'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data['Ranking'].value_counts()>10).sum())
data.loc[:, ['Ranking']].info()

Итого у нас 12975 уникальных значений в колонке с номером места ресторана среди остальных по своему городу и всего 50000 строк. Пустые значения отсутствуют.

#### Возьмем признак "Rating" - рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель)

In [ ]:
display(data['Rating'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data['Rating'].value_counts()>10).sum())
data.loc[:, ['Rating']].info()

In [ ]:
data.Rating.nunique()

Итого у нас 10 уникальных значений в колонке с рейтингом и всего 50000 строк. Пустые значения отсутствуют. Столбец числовой.

#### Вернёмся к колонке 'Reviews' - данные о 2-х отзывах, которые отображаются на сайте ресторана

In [ ]:
display(data['Reviews'].value_counts())
data.loc[:, ['Reviews']].info()

Столбец строковый, без пропусков. 41858 уникальных значений.

#### Вернёмся к колонке 'Number of Reviews' - количество отзывов о ресторане

In [ ]:
display(data['Number of Reviews'].value_counts())
data.loc[:, ['Number of Reviews']].info()

Итого у нас 1574 уникальных значений в колонке с количеством отзывов и всего 50000 строк. Столбец числовой, без пропусков.

#### Возьмем признак "ID_TA" - идентификатор ресторана в БД TripAdvisor

In [ ]:
display(data['ID_TA'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data['ID_TA'].value_counts()>10).sum())
data.loc[:, ['ID_TA']].info()

Итого у нас 49963 уникальных значений и всего 50000 строк. Столбец строковый, пропуски отсутствуют.

#### Возьмем признак "Restaurant_id" - идентификационный номер ресторана / сети ресторанов

In [ ]:
display(data['Restaurant_id'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data['Restaurant_id'].value_counts()>10).sum())
data.loc[:, ['Restaurant_id']].info()

Итого у нас 13094 уникальных значений и всего 50000 строк. Столбец строковый, пропуски отсутствуют.

#### Вернёмся к колонке 'Cuisine_Style' - кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане

In [ ]:
display(data['Cuisine_Style'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data['Cuisine_Style'].value_counts()>10).sum())
data.loc[:, ['Cuisine_Style']].info()

Итого у нас 10731 уникальных значений и всего 50000 строк. Столбец строковый, содержит пропуски - 11590 строк. Преобразуем колонку

In [ ]:
data[['Cuisine_Style', 'URL_TA']][data.Cuisine_Style.isna() == True].sample(5)

Используем функцию для преобразования колонки 'cuisine_style' (поиск информации проводим по столбцу 'url_ta')

In [ ]:
data['Cuisine_Style'][data.Cuisine_Style.isna() == True] = data.apply(lambda row: cuis_style(row), axis=1) 

In [ ]:
data.loc[:, ['Cuisine_Style']].info()

Количество ненулевых значений увеличилось (было 38410, стало 38538).

Применим следующую функцию для преобразования колонки 'cuisine_style' (поиск информации проводим по столбцу 'url_ta')

In [ ]:
data['Cuisine_Style'][data.Cuisine_Style.isna() == True] = data.apply(lambda row: cuis(row), axis=1)

In [ ]:
data.loc[:, ['Cuisine_Style']].info()

Количество ненулевых значений увеличилось (было 38538, стало 45282). 

Применим следующую функцию для преобразования колонки 'cuisine_style' (поиск информации проводим по столбцу 'reviews')

In [ ]:
data[['Cuisine_Style', 'Reviews']][data.Cuisine_Style.isna() == True].sample(5)

In [ ]:
data['Cuisine_Style'][data.Cuisine_Style.isna() == True] = data.apply(lambda row: f_cuis_style(row), axis=1)

In [ ]:
data.loc[:, ['Cuisine_Style']].info()
data.Cuisine_Style.isna().sum()

Количество ненулевых значений увеличилось (было 45282, стало 45539). Продолжаем преобразование

In [ ]:
data['Cuisine_Style'][data.Cuisine_Style.isna() == True] = data.apply(lambda row: f_cuis(row), axis=1)

In [ ]:
data.loc[:, ['Cuisine_Style']].info()
data.Cuisine_Style.isna().sum()

Количество ненулевых значений увеличилось (было 45539, стало 46278).

In [ ]:
data.head(10)

In [ ]:
data.index

In [ ]:
# Создадим столбец index для дальнейшей группировки и отдельный датафрейм data_cuis_style_top для определения значения, 
# встречающегося чаще других. Продолжаем преобразование (применяем функцию fix_cuisine_style)
data['index'] = [i for i in range(len(data))]

In [ ]:
data_cuis_style_top = data.assign(Cuisine_Style = data.Cuisine_Style.apply(fix_cuisine_style).str.split(",")).explode("Cuisine_Style")

In [ ]:
display(data_cuis_style_top['Cuisine_Style'].value_counts())
print("Значений, встретившихся в столбце более 10 раз:", (data_cuis_style_top['Cuisine_Style'].value_counts()>10).sum())
data_cuis_style_top.loc[:, ['Cuisine_Style']].info()

Итого у нас 126 уникальных значений и всего 131266 строк. Столбец строковый, содержит пропуски - 3722 строки. Продолжим преобразование.

In [ ]:
# Сократим количество уникальных значений в признаке
cuis_style_with_freqs = list(data_cuis_style_top.Cuisine_Style.value_counts())

In [ ]:
top_cuis_style_count = int(np.percentile(cuis_style_with_freqs, 25))
top_cuis_style_count

In [ ]:
cuis_style_to_throw_away = list(data_cuis_style_top.Cuisine_Style.value_counts()\
                                [data_cuis_style_top.Cuisine_Style.value_counts() < top_cuis_style_count].index)
cuis_style_to_throw_away

In [ ]:
data_cuis_style_top.loc[data_cuis_style_top['Cuisine_Style'].isin(cuis_style_to_throw_away), 'Cuisine_Style'] = 'other_cuis_style'

In [ ]:
data_cuis_style_top.shape

In [ ]:
# Продолжим преобразование
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'VeganOptions'", "'Healthy'", "'GlutenFreeOptions'"], 
                                                                                 "'VegetarianFriendly'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Italian'", "'Spanish'", "'Portuguese'", "'Greek'", 
                                                                                  "'Turkish'", "'Moroccan'"], "'Mediterranean'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Pizza'", "'StreetFood'"], "'FastFood'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Japanese'", "'Sushi'", "'Chinese'", "'Thai'", 
                                                                                  "'Vietnamese'", "'Korean'", "'Indonesian'", 
                                                                                  "'Malaysian'", "'Tibetan'", "'Taiwanese'"], "'Asian'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Pub'", "'WineBar'", "'Gastropub'", "'BrewPub'"], 
                                                                                 "'Bar'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace("'Steakhouse'", "'Cafe'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Austrian'", "'Hungarian'", "'German'", "'Polish'", 
                                                                                  "'Slovenian'", "'Swiss'", "'Czech'", "'Belgian'", 
                                                                                  "'Croatian'"], "'CentralEuropean'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Halal'", "'Lebanese'", "'Israeli'", "'Persian'", 
                                                                                  "'Arabic'", "'Kosher'"], "'MiddleEastern'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Barbecue'", "'Grill'"], "'American'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Latin'", "'Argentinean'", "'Brazilian'", 
                                                                                  "'Peruvian'", "'Venezuelan'"], "'SouthAmerican'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Danish'", "'Swedish'", "'Norwegian'"], "'Scandinavian'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Caribbean'", "'Jamaican'", "'Cuban'"], 
                                                                                 "'CentralAmerican'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Delicatessen'", "'French'"], "'European'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Pakistani'", "'Nepali'", "'Bangladeshi'", 
                                                                                  "'SriLankan'"], "'Indian'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Ethiopian'", "'Tunisian'"], "'African'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace("'Contemporary'", "'Fusion'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Soups'", "'Balti'", "'Russian'", "'Afghani'"], 
                                                                                 "'EasternEuropean'")
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'British'", "'Dutch'", "'Irish'", "'Scottish'"], 
                                                                                 "'NorthWesternEuropean'")

In [ ]:
data_cuis_style_top.shape

In [ ]:
# Количество уникальных значений в признаке cuisine_style сократилось до (было 126):
data_cuis_style_top['Cuisine_Style'].nunique()

In [ ]:
data_cuis_style_top.Cuisine_Style.isna().sum()

3722 строки содержит пропуски.

Заполним оставшиеся пропуски значением, встречающимся чаще остальных.

In [ ]:
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top['Cuisine_Style'].fillna(data_cuis_style_top['Cuisine_Style'].describe().top)

In [ ]:
data_cuis_style_top['Cuisine_Style'].value_counts()

In [ ]:
data_cuis_style_top['Cuisine_Style'].nunique()

In [ ]:
# Продолжим сокращать количество уникальных значений
cuis_style_with_freqs_sec = list(data_cuis_style_top.Cuisine_Style.value_counts())
top_cuis_style_count_sec = int(np.percentile(cuis_style_with_freqs_sec, 10))
top_cuis_style_count_sec

In [ ]:
cuis_style_to_throw_away_sec = list(data_cuis_style_top.Cuisine_Style.\
                                    value_counts()[data_cuis_style_top.Cuisine_Style.value_counts() < top_cuis_style_count_sec].index)
cuis_style_to_throw_away_sec

In [ ]:
data_cuis_style_top.loc[data_cuis_style_top['Cuisine_Style'].isin(cuis_style_to_throw_away_sec), 'Cuisine_Style'] = 'other_cuisine_style'

In [ ]:
data_cuis_style_top['Cuisine_Style'].nunique()

Количество уникальных значений сократилось с 26 до 24. Преобразуем далее колонку

In [ ]:
data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Diner'", "other_cuis_style", "'CentralAmerican'"], 
                                                                                 "other_cuisine_style")

In [ ]:
# Перед созданием дамми-признаков, создадим копию столбца Cuisine_Style
data_cuis_style_top['cuisine_style'] = data_cuis_style_top['Cuisine_Style'].copy()

In [ ]:
data_cuis_style_top['cuisine_style'].describe().top

In [ ]:
# используем далее для заполнения пропусков в датафрейме data столбца Cuisine_Style
top_cuis = data_cuis_style_top['cuisine_style'].describe().top

In [ ]:
data_cuis_style_top = pd.get_dummies(data_cuis_style_top, prefix = 'Cuis_Style', prefix_sep = '_',\
                                     columns = ['Cuisine_Style'], dummy_na=True)

In [ ]:
data_cuis_style_top.info()

Попробуем создать новый признак, создадим новый датафрейм data_cuis_style_new. Проведем его преобразование.

In [ ]:
data_cuis_style_new = pd.DataFrame(data_cuis_style_top.groupby(['index']).sum())
data_cuis_style_new

В датафрейме 50000 строк. Заменим все числа больше 0 на 1, а равные 0 на 0 (После группировки датафрейм data_cuis_style_top по index, мы получили общее количество дубликатов по каждой дамми-переменной признака 'тип кухни'. Там, где индекс был разбит на две строки со значением дамми-переменной равной 1, при группировке по индексу дамми-переменная стала равна 2, и т.д.). Преобразуем датафрейм.  

In [ ]:
data_cuis_style_new.columns.tolist()

In [ ]:
data_cuis_style_new['index'] = [i for i in range(len(data_cuis_style_new))]

In [ ]:
data_cuis_style_new = data_cuis_style_new[['index', "Cuis_Style_'African'", "Cuis_Style_'American'", "Cuis_Style_'Asian'", "Cuis_Style_'Bar'", 
                                           "Cuis_Style_'Cafe'", "Cuis_Style_'CentralEuropean'", "Cuis_Style_'EasternEuropean'", 
                                           "Cuis_Style_'European'", "Cuis_Style_'FastFood'", "Cuis_Style_'Fusion'", "Cuis_Style_'Indian'", 
                                           "Cuis_Style_'International'", "Cuis_Style_'Mediterranean'", "Cuis_Style_'Mexican'", 
                                           "Cuis_Style_'MiddleEastern'", "Cuis_Style_'NorthWesternEuropean'", "Cuis_Style_'Scandinavian'", 
                                           "Cuis_Style_'Seafood'", "Cuis_Style_'SouthAmerican'", "Cuis_Style_'VegetarianFriendly'", 
                                           'Cuis_Style_other_cuisine_style', 'Ranking']]

In [ ]:
data_cuis_style_new.dtypes

In [ ]:
data_cuis_style_new.head(7)

In [ ]:
def dummies(x):
    if x >= 1:
         return 1
    else:
         return 0

for x in ["Cuis_Style_'African'", "Cuis_Style_'American'", "Cuis_Style_'Asian'", "Cuis_Style_'Bar'", 
          "Cuis_Style_'Cafe'", "Cuis_Style_'CentralEuropean'", "Cuis_Style_'EasternEuropean'", 
          "Cuis_Style_'European'", "Cuis_Style_'FastFood'", "Cuis_Style_'Fusion'", "Cuis_Style_'Indian'", 
          "Cuis_Style_'International'", "Cuis_Style_'Mediterranean'", "Cuis_Style_'Mexican'", 
          "Cuis_Style_'MiddleEastern'", "Cuis_Style_'NorthWesternEuropean'", "Cuis_Style_'Scandinavian'", 
          "Cuis_Style_'Seafood'", "Cuis_Style_'SouthAmerican'", "Cuis_Style_'VegetarianFriendly'", 
          'Cuis_Style_other_cuisine_style']:
    data_cuis_style_new[x] = data_cuis_style_new[x].apply(lambda x: dummies(x))
data_cuis_style_new.head()

In [ ]:
data_cuis_style_new['Ranking_check'] = data_cuis_style_new['Ranking']
data_cuis_style_new = data_cuis_style_new.drop(['index', 'Ranking'], axis = 1)

In [ ]:
data_cuis_style_new.head()

In [ ]:
data = data.merge(data_cuis_style_new, how='inner', on='index')

In [ ]:
data.info()

In [ ]:
data = data.drop(['index'], axis=1)

Мы определили, что чаще всех встречается кухня 'VegetarianFriendly'. заполним пропуски в датафрейме data.

In [ ]:
data.loc[:, ['Cuisine_Style']].info()
data.Cuisine_Style.isna().sum()

In [ ]:
data['Cuisine_Style'] = data['Cuisine_Style'].fillna(top_cuis)

In [ ]:
data.loc[:, ['Cuisine_Style']].info()
data.Cuisine_Style.isna().sum()

Как видим, все пропуски заполнены

In [ ]:
data.info()

Как видим, пустых значений не осталось. 

In [ ]:
# Перед созданием дамми-признаков, создадим копию столбца city
data['city'] = data['City'].copy()

Создадим dummy-признаки для нескольких переменных:

In [ ]:
data = pd.get_dummies(data, prefix = 'City', prefix_sep = '_', columns = ['City'], dummy_na=True)

In [ ]:
data = pd.get_dummies(data, prefix = 'Price_Range', prefix_sep = '_', columns = ['Price Range'], dummy_na=True)

In [ ]:
data.info()

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака 'Ranking'

In [ ]:
data.Ranking.describe()

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
data['Ranking'].hist(bins=100)

In [ ]:
sns.boxplot(data.Ranking, color = 'yellow')

Распределение мест в диапазоне 25-75% - от 972 до 5241. Медианное значение - 2278.

In [ ]:
IQR_col_num('Ranking')
data.Ranking.hist()
data['Ranking'].loc[data['Ranking'].between(describe_col_IQR_l('Ranking'), 
                                        describe_col_IQR_r('Ranking'))].hist(bins = 10, range = (0, 20000), label = 'IQR')
plt.legend()

Выбросов нет, возможно присутствуют аномальные значения.

Сократим количество уникальных значений в признаке, создадим новый признак 'ranking_range'. Применим функцию, группирующую значения столбца 'ranking' по заданному условию и возвращающую значения от 1 до 5.

In [ ]:
data['Ranking_range'] = data.Ranking.apply(rank_range)

In [ ]:
data.loc[:, ['Ranking_range']].info()

Cтолбец числовой и без пропусков. Посмотрим на его распределение:

In [ ]:
data.Ranking_range.describe()

In [ ]:
IQR_col_num('Ranking_range')
data.Ranking_range.hist()
data['Ranking_range'].loc[data['Ranking_range'].between(describe_col_IQR_l('Ranking_range'), 
                                        describe_col_IQR_r('Ranking_range'))].hist(bins = 10, range = (0, 10), label = 'IQR')
plt.legend()

In [ ]:
sns.boxplot(data.Ranking_range, color = 'orange')

In [ ]:
# Создадим новые признаки
data['Ranking_range'] = np.sqrt(data.Ranking_range)**(1/30)

In [ ]:
data.Ranking_range.hist(bins=15)

In [ ]:
data['Ranking'][data['city'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

In [ ]:
data['c_London'] = data['Ranking'][data['city'] =='London']

In [ ]:
data.c_London.hist(bins=100)

In [ ]:
data.c_London.value_counts()

In [ ]:
data.c_London.mean()

In [ ]:
data['c_London'] = data['c_London'].fillna(data.c_London.mean())

In [ ]:
data.c_London.hist(bins=7)

In [ ]:
data['c_Paris'] = data['Ranking'][data['city'] =='Paris']
data['c_Rome'] = data['Ranking'][data['city'] =='Rome']
data['c_Barcelona'] = data['Ranking'][data['city'] =='Barcelona']
data['c_Milan'] = data['Ranking'][data['city'] =='Milan']
data['c_Madrid'] = data['Ranking'][data['city'] =='Madrid']
data['c_other_city'] = data['Ranking'][data['city'] =='other_city']
data['c_Berlin'] = data['Ranking'][data['city'] =='Berlin']
data['c_Prague'] = data['Ranking'][data['city'] =='Prague']
data['c_Lisbon'] = data['Ranking'][data['city'] =='Lisbon']
data['c_Amsterdam'] = data['Ranking'][data['city'] =='Amsterdam']
data['c_Vienna'] = data['Ranking'][data['city'] =='Vienna']
data['c_Stockholm'] = data['Ranking'][data['city'] =='Stockholm']
data['c_Zurich'] = data['Ranking'][data['city'] =='Zurich']
data['c_Edinburgh'] = data['Ranking'][data['city'] =='Edinburgh']
data['c_Copenhagen'] = data['Ranking'][data['city'] =='Copenhagen']
data['c_Brussels'] = data['Ranking'][data['city'] =='Brussels']

data['c_Paris'] = data['c_Paris'].fillna(data.c_Paris.mean())
data['c_Rome'] = data['c_Rome'].fillna(data.c_Rome.mean())
data['c_Barcelona'] = data['c_Barcelona'].fillna(data.c_Barcelona.mean())
data['c_Milan'] = data['c_Milan'].fillna(data.c_Milan.mean())
data['c_Madrid'] = data['c_Madrid'].fillna(data.c_Madrid.mean())
data['c_other_city'] = data['c_other_city'].fillna(data.c_other_city.mean())
data['c_Berlin'] = data['c_Berlin'].fillna(data.c_Berlin.mean())
data['c_Prague'] = data['c_Prague'].fillna(data.c_Prague.mean())
data['c_Lisbon'] = data['c_Lisbon'].fillna(data.c_Lisbon.mean())
data['c_Amsterdam'] = data['c_Amsterdam'].fillna(data.c_Amsterdam.mean())
data['c_Vienna'] = data['c_Vienna'].fillna(data.c_Vienna.mean())
data['c_Stockholm'] = data['c_Stockholm'].fillna(data.c_Stockholm.mean())
data['c_Zurich'] = data['c_Zurich'].fillna(data.c_Zurich.mean())
data['c_Edinburgh'] = data['c_Edinburgh'].fillna(data.c_Edinburgh.mean())
data['c_Copenhagen'] = data['c_Copenhagen'].fillna(data.c_Copenhagen.mean())
data['c_Brussels'] = data['c_Brussels'].fillna(data.c_Brussels.mean())

In [ ]:
data.info()

### Посмотрим распределение целевой переменной

In [ ]:
data['Rating'].value_counts(ascending=True).plot(kind='barh')

In [ ]:
data.Rating.describe()

Можно заметить, что размах параметра находится в границах от 1 до 5 включительно (min = 1, max = 5). Нулевые значения мы присваивали, чтобы различать train и test.

In [ ]:
IQR_col_num('Rating')
data.Rating.hist()
data['Rating'].loc[data['Rating'].between(describe_col_IQR_l('Rating'), 
                                        describe_col_IQR_r('Rating'))].hist(bins = 10, range = (0, 6), label = 'IQR')
plt.legend()

Выбросы отсутствуют. 

#### Посмотрим распределение переменной 'Number of Reviews'

In [ ]:
data['Number of Reviews'].describe()

In [ ]:
data['Number of Reviews'].hist(bins=25)

Распределение количества отзывов в диапазоне 25-75% - от 8 до 105. Медианное значение - 31 отзывов.

In [ ]:
IQR_col_num('Number of Reviews')
data['Number of Reviews'].hist()
data['Number of Reviews'].loc[data['Number of Reviews'].between(describe_col_IQR_l('Number of Reviews'), 
                                        describe_col_IQR_r('Number of Reviews'))].hist(bins = 10, 
                                                                                     range = (0, 10000), label = 'IQR')
plt.legend()

In [ ]:
sns.boxplot(data['Number of Reviews'], color = 'green')

Возможно присутствуют аномальные значения.

Сократим количество уникальных значений в признаке, создадим новый признак 'numb_rev_range'. Применим функцию, группирующую значения столбца 'numb_of_reviews' по заданному условию и возвращающую значения от 1 до 5.

In [ ]:
%%time
data['numb_rev_range'] = data['Number of Reviews'].apply(numb_review_range)

In [ ]:
data.numb_rev_range.describe()

Cоздадим новые признаки: 'numb_rev_1/30' и 'numb_rev_range'

In [ ]:
(np.sqrt(data.numb_rev_range[data.numb_rev_range > 0])**(1/30)).hist(bins=15)

In [ ]:
data['numb_rev_1/30'] = np.sqrt(data.numb_rev_range[data.numb_rev_range > 0])**(1/30)

In [ ]:
(np.sqrt(data.numb_rev_range[data.numb_rev_range > 0])**1.7).hist(bins=15)

In [ ]:
data['numb_rev_range'] = np.sqrt(data.numb_rev_range[data.numb_rev_range > 0])**1.7

In [ ]:
data.info()

Новые признаки числовые и не имеют пустых значений.

#### Создадим новые признаки для переменной 'ID_TA'

In [ ]:
display(data.ID_TA.value_counts())
data.loc[:, ['ID_TA']].info()

Столбец строковый, пропуски отсутствуют. Можно заметить, что id находятся в диапазоне от 1 до 2. Преобразуем колонку и создадим новый признак 'id_ta_new'.

In [ ]:
data['id_ta_new'] = data['ID_TA']

In [ ]:
for k in range(data.ID_TA.value_counts()[0]+1):
    ID_TA_k = list(data.ID_TA.value_counts()[data.ID_TA.value_counts() == k].index)
    data.loc[data['id_ta_new'].isin(ID_TA_k), 'id_ta_new'] = k

#### Создадим новые признаки для переменной 'Restaurant_id'

In [ ]:
display(data.Restaurant_id.value_counts())
data.loc[:, ['Restaurant_id']].info()

Столбец строковый, пропуски отсутствуют. Можно заметить, что id находится в диапазоне от 1 до 19. Преобразуем колонку и создадим новый признак 'restaurant_id_new'

In [ ]:
data['restaurant_id_new'] = data['Restaurant_id']

In [ ]:
for i in range(data.Restaurant_id.value_counts()[0]+1):
    restaurant_id_i = list(data.Restaurant_id.value_counts()[data.Restaurant_id.value_counts() == i].index)
    data.loc[data['restaurant_id_new'].isin(restaurant_id_i), 'restaurant_id_new'] = i

In [ ]:
display(data.restaurant_id_new.value_counts())

In [ ]:
data.head()

In [ ]:
data.loc[:, ['restaurant_id_new']].info()

#### Создадим dummy-признаки для новых переменных:

In [ ]:
data = pd.get_dummies(data, prefix = 'id_ta_new', prefix_sep = '_', columns = ['id_ta_new'], dummy_na=True)

In [ ]:
data = pd.get_dummies(data, prefix = 'restaurant_id_new', prefix_sep = '_', columns = ['restaurant_id_new'], dummy_na=True)

In [ ]:
data.isna().sum().describe()

Пустых значений нет

#### Построим матрицу корреляций:

In [ ]:
corr = data.drop(['Restaurant_id', 'Reviews', 'Cuisine_Style', 'URL_TA', 'ID_TA', 'sample'], axis=1).dropna(axis=1).corr()
corr

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    # Переименуем столбец Cuisine Style для удобства
    df_input.columns = (['Restaurant_id', 'City', 'Cuisine_Style', 'Ranking', 'Price Range',
       'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA', 'sample', 'Rating'])
    
    df_output = df_input.copy()
    
    def fix_cuisine_style(x):        # функция для преобразования колонки 'cuisine_style'
        if type(x) != float:
            if x == None:
                return None
            x = x.replace(' ', '')
            x = x.replace('[', '')
            x = x.replace(']', '')
            return x
    
    def fix_reviews(x):     # функция для преобразования колонки 'reviews'
        if pd.isnull(x):
            return x
        if x == 'nan':
            return None
        x = x.replace('[[', '')
        x = x.replace(']]', '')
        x = x.replace('[', '')
        x = x.replace(']', '')
        return x

    def rew(x):      # функция замены символа по указанному шаблону на 'None'
        if pd.isnull(x):
            return x
        if x == 'nan':
            return None
        rew = re.sub(r'^\S\s$', 'None', str(x))
        return rew
    
    def describe_col_IQR_l(column):              # функция, возвращающая значение левой границы выброса
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        l = Q1 - 1.5*IQR
        return l


    def describe_col_IQR_r(column):              # функция, возвращающая значение правой границы выброса
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        r = Q3 + 1.5*IQR
        return r

    def rank_range(x):                                                                # функция, группирующая значения столбца 'ranking'
        if df_output['Ranking'].min() <= x < df_output['Ranking'].quantile(0.25):         # по заданному условию и возвращающая значения [1-5]
            return 1
        elif df_output['Ranking'].quantile(0.25) <= x < df_output['Ranking'].quantile(0.50):
            return 2
        elif df_output['Ranking'].quantile(0.50) <= x < df_output['Ranking'].quantile(0.75):
            return 3
        elif df_output['Ranking'].quantile(0.75) <= x <= describe_col_IQR_r('Ranking'):
            return 4
        elif x > describe_col_IQR_r('Ranking'):   # x > (Q3 + 1.5*IQR)
            return 5
        return None

    def numb_review_range(x):                                                                          # функция, группирующая значения столбца 
        if df_output['Number of Reviews'].min() <= x < df_output['Number of Reviews'].quantile(0.25):      # 'numb_of_reviews' и возвращающая значения [1-5]
            return 1
        elif df_output['Number of Reviews'].quantile(0.25) <= x < df_output['Number of Reviews'].quantile(0.50):
            return 2
        elif df_output['Number of Reviews'].quantile(0.50) <= x < df_output['Number of Reviews'].quantile(0.75):
            return 3
        elif df_output['Number of Reviews'].quantile(0.75) <= x <= describe_col_IQR_r('Number of Reviews'):
            return 4
        elif x > describe_col_IQR_r('Number of Reviews'):   # x > (Q3 + 1.5*IQR)
            return 5
        return None

    def cuis_style(row):   # функция для преобразования колонки 'cuisine_style' (поиск по столбцу 'url_ta')
        for word in ['Italian', 'Spanish', 'Portuguese', 'Greek', 'Turkish', 'Moroccan', 'Mediterranean']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Mediterranean'"
                return row['Cuisine_Style']
        for word in ['Japanese', 'Chinese', 'Thai', 'Vietnamese', 'Korean', 'Indonesian', 'Asian', 'Filipino']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Asian'"
                return row['Cuisine_Style']
        for word in ['Scandinavian']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Scandinavian'"
                return row['Cuisine_Style']
        for word in ['Indian']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Indian'"
                return row['Cuisine_Style']    
        for word in ['French']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'European'"
                return row['Cuisine_Style']    
        for word in ['Mexican']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Mexican'"
                return row['Cuisine_Style']
        return None

    def cuis(row):   # функция для преобразования колонки 'cuisine_style' (поиск по столбцу 'url_ta')
        for word in ['Ital', 'Spanis', 'Greek', 'Turkish', 'Moroc', 'Mediter', 'Neap', 'Milan', 'pasta', 
                     'Rome', 'Madrid', 'tapas', 'catalan', 'ravioli']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Mediterranean'"
                return row['Cuisine_Style']
        for word in ['Japan', 'Sush', 'Chines', 'Thai', 'Vietna', 'Korea', 'Indones', 'Malays', 'Tibet', 'Taiwan', 'Asia', 
                     'ramen']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Asian'"
                return row['Cuisine_Style']
        for word in ['Austri', 'Hungari', 'Germ', 'Polish', 'Sloven', 'Swiss', 'Czech', 'Belgi', 'Croat', 
                     'Amsterd', 'bavarian', 'Hamburg', 'Zurich']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'CentralEuropean'"
                return row['Cuisine_Style']    
        for word in ['Hala', 'Leban', 'Isra', 'Pers', 'Arab', 'Kosher', 'MiddleEast', 'MidEast', 'falafel']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'MiddleEastern'"
                return row['Cuisine_Style']   
        for word in ['Latin', 'Argent', 'Brazil', 'Peru', 'Venez', 'SouthAmer']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'SouthAmerican'"
                return row['Cuisine_Style']    
        for word in ['Dani', 'Swed', 'Norw', 'Scand']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Scandinavian'"
                return row['Cuisine_Style']
        for word in ['Pakist', 'Nepal', 'Banglad', 'SriLan', 'Indi', 'masala']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Indian'"
                return row['Cuisine_Style']    
        for word in ['Delicatessen', 'French', 'Croissan', 'BAGUETT']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'European'"
                return row['Cuisine_Style']    
        for word in ['Carib', 'Jamai', 'Cuba', 'CentralAmer']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'CentralAmerican'"
                return row['Cuisine_Style']
        for word in ['Barbec', 'Gril', 'BBQ']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'American'"
                return row['Cuisine_Style']
        for word in ['Ethiop', 'Tunis', 'Afri', 'Nigerian']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'African'"
                return row['Cuisine_Style']    
        for word in ['Soup', 'Balt', 'Russ', 'Afghan', 'EasternEu']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'EasternEuropean'"
                return row['Cuisine_Style']   
        for word in ['Brit', 'Dutch', 'Irish', 'Scotti', 'Dublin', 'Portug', 'Lisbon']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'NorthWesternEuropean'"
                return row['Cuisine_Style']
        for word in ['Contemp', 'Fusi']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Fusion'"
                return row['Cuisine_Style']   
        for word in ['Steakh', 'Cafe', 'Caffe', 'buffet', 'gelato']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Cafe'"
                return row['Cuisine_Style']   
        for word in ['Pizz', 'StreetFo', 'FastF', 'Pitstop', 'burger', 'fries', 'bistro', 'kebab', 'sandwich', 
                     'street foo', 'KFC', 'donalds', 'fast foo']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'FastFood'"
                return row['Cuisine_Style']
        for word in ['Pub', 'WineBa', 'Gastropu', 'BrewPu', 'Bar', 'cocktail', 'football', 'beer']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'Bar'"
                return row['Cuisine_Style']
        for word in ['Veg', 'Health', 'GlutenFre', 'Gluten Fre']:
            if word.lower() in row.URL_TA.lower():
                row['Cuisine_Style'] = "'VegetarianFriendly'"
                return row['Cuisine_Style']
        return None

    def f_cuis_style(row):   # функция для преобразования колонки 'cuisine_style' (поиск по столбцу 'reviews')
        for word in ['Italian', 'Spanish', 'Portuguese', 'Greek', 'Turkish', 'Moroccan', 'Mediterranean']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Mediterranean'"
                return row['Cuisine_Style']
        for word in ['Japanese', 'Chinese', 'Thai', 'Vietnamese', 'Korean', 'Indonesian', 'Asian', 'Filipino']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Asian'"
                return row['Cuisine_Style']
        for word in ['Scandinavian']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Scandinavian'"
                return row['Cuisine_Style']
        for word in ['Indian']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Indian'"
                return row['Cuisine_Style']    
        for word in ['French']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'European'"
                return row['Cuisine_Style']    
        for word in ['Mexican']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Mexican'"
                return row['Cuisine_Style']
        return None

    def f_cuis(row):   # функция для преобразования колонки 'cuisine_style' (поиск по столбцу 'reviews')
        for word in ['Ital', 'Spanis', 'Greek', 'Turkish', 'Moroc', 'Mediter', 'Neap', 'Milan', 'pasta', 
                     'Rome', 'Madrid', 'tapas', 'catalan', 'ravioli']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Mediterranean'"
                return row['Cuisine_Style']
        for word in ['Japan', 'Sush', 'Chines', 'Thai', 'Vietna', 'Korea', 'Indones', 'Malays', 'Tibet', 'Taiwan', 'Asia', 
                     'ramen']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Asian'"
                return row['Cuisine_Style']
        for word in ['Austri', 'Hungari', 'Germ', 'Polish', 'Sloven', 'Swiss', 'Czech', 'Belgi', 'Croat', 
                     'Amsterd', 'bavarian', 'Hamburg', 'Zurich']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'CentralEuropean'"
                return row['Cuisine_Style']    
        for word in ['Hala', 'Leban', 'Isra', 'Pers', 'Arab', 'Kosher', 'MiddleEast', 'MidEast', 'falafel']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'MiddleEastern'"
                return row['Cuisine_Style']   
        for word in ['Latin', 'Argent', 'Brazil', 'Peru', 'Venez', 'SouthAmer']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'SouthAmerican'"
                return row['Cuisine_Style']    
        for word in ['Dani', 'Swed', 'Norw', 'Scand']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Scandinavian'"
                return row['Cuisine_Style']
        for word in ['Pakist', 'Nepal', 'Banglad', 'SriLan', 'Indi', 'masala']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Indian'"
                return row['Cuisine_Style']    
        for word in ['Delicatessen', 'French', 'Croissan', 'BAGUETT']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'European'"
                return row['Cuisine_Style']    
        for word in ['Carib', 'Jamai', 'Cuba', 'CentralAmer']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'CentralAmerican'"
                return row['Cuisine_Style']
        for word in ['Barbec', 'Gril', 'BBQ']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'American'"
                return row['Cuisine_Style']
        for word in ['Ethiop', 'Tunis', 'Afri', 'Nigerian']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'African'"
                return row['Cuisine_Style']    
        for word in ['Soup', 'Balt', 'Russ', 'Afghan', 'EasternEu']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'EasternEuropean'"
                return row['Cuisine_Style']   
        for word in ['Brit', 'Dutch', 'Irish', 'Scotti', 'Dublin', 'Portug', 'Lisbon']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'NorthWesternEuropean'"
                return row['Cuisine_Style']
        for word in ['Contemp', 'Fusi']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Fusion'"
                return row['Cuisine_Style']   
        for word in ['Steakh', 'Cafe', 'Caffe', 'buffet', 'gelato']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Cafe'"
                return row['Cuisine_Style']   
        for word in ['Pizz', 'StreetFo', 'FastF', 'Pitstop', 'burger', 'fries', 'bistro', 'kebab', 'sandwich', 
                     'street foo', 'KFC', 'donalds', 'fast foo']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'FastFood'"
                return row['Cuisine_Style']
        for word in ['Pub', 'WineBa', 'Gastropu', 'BrewPu', 'Bar', 'cocktail', 'football', 'beer']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'Bar'"
                return row['Cuisine_Style']
        for word in ['Veg', 'Health', 'GlutenFre', 'Gluten Fre']:
            if word.lower() in row.Reviews.lower():
                row['Cuisine_Style'] = "'VegetarianFriendly'"
                return row['Cuisine_Style']
        return None
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
        
    
    # ################### 2. NAN ############################################################## 
    # перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    df_output['Price_Range_isNAN'] = pd.isna(df_output['Price Range']).astype('uint8')
    df_output['Reviews_isNAN'] = pd.isna(df_output['Reviews']).astype('uint8')
    df_output['Cuisine_Style_isNAN'] = pd.isna(df_output['Cuisine_Style']).astype('uint8')
    # Далее произведём предобработку столбца Reviews и заполним пропуски
    df_output['Reviews'] = df_output['Reviews'].apply(fix_reviews)
    df_output['Reviews'] = df_output['Reviews'].apply(rew)
    # Заменим пропуски на 0 там, где в колонке 'Reviews' нет отзывов 
    # а в остальных случаях заменим пропуски на значение медианы:
    def fillna_numb_rev(row):  
        if row.Reviews == 'None':
            return 0
        else:
            return df_output['Number of Reviews'].quantile(0.50)
    df_output['Number of Reviews'][df_output['Number of Reviews'].\
                                   isna() == True] = df_output.apply(lambda row: fillna_numb_rev(row), axis=1)
    # Заменим пропуски на значение среднего ценового сегмента
    df_output['Price Range'] = df_output['Price Range'].fillna(df_output['Price Range'].describe().top)
    # Заменим пропуски и None на 'zero' там, где в колонке 'Number of Reviews' нет отзывов ('Number of Reviews' == 0) 
    # а в остальных случаях заменим пропуски на значение 'not good, not bad':
    def fillna_rev(row):  
        if row['Number of Reviews'] == 0:
            return 'zero'
        else:
            return 'not good, not bad'
    df_output['Reviews'][df_output['Reviews'].isna() == True] = df_output.apply(lambda row: fillna_rev(row), axis=1)
    df_output['Reviews'][df_output['Reviews'] == 'None'] = df_output.apply(lambda row: fillna_rev(row), axis=1)
    # Пропуски 'Cuisine_Style' заполним ниже, после обработки данного столбца
    
    
    # ################### 3. Encoding ############################################################## 
    # Возьмем признак "City"
    # Сократим количество уникальных значений в признаке
    cities_with_freqs = list(df_output.City.value_counts())
    top_cities_count = int(np.percentile(cities_with_freqs, 25))
    cities_to_throw_away = list(df_output.City.value_counts()[df_output.City.value_counts() < top_cities_count].index)
    df_output.loc[df_output['City'].isin(cities_to_throw_away), 'City'] = 'other_city'
    # Вернёмся к колонке 'Cuisine_Style'
    # Используем функции для преобразования колонки 'cuisine_style' (поиск информации проводим по столбцу 'url_ta')
    df_output['Cuisine_Style'][df_output.Cuisine_Style.isna() == True] = df_output.apply(lambda row: cuis_style(row), axis=1)
    df_output['Cuisine_Style'][df_output.Cuisine_Style.isna() == True] = df_output.apply(lambda row: cuis(row), axis=1)
    # Используем функции для преобразования колонки 'cuisine_style' (поиск информации проводим по столбцу reviews')
    df_output['Cuisine_Style'][df_output.Cuisine_Style.isna() == True] = df_output.apply(lambda row: f_cuis_style(row), axis=1)
    df_output['Cuisine_Style'][df_output.Cuisine_Style.isna() == True] = df_output.apply(lambda row: f_cuis(row), axis=1)
    # Создадим столбец index для дальнейшей группировки и отдельный датафрейм data_cuis_style_top для определения значения, 
    # встречающегося чаще других. Продолжаем преобразование (применяем функцию fix_cuisine_style)
    df_output['index'] = [i for i in range(len(df_output))]
    data_cuis_style_top = df_output.assign(Cuisine_Style = df_output.Cuisine_Style.apply(fix_cuisine_style).str.split(",")).explode("Cuisine_Style")
    # Сократим количество уникальных значений в признаке
    cuis_style_with_freqs = list(data_cuis_style_top.Cuisine_Style.value_counts())
    top_cuis_style_count = int(np.percentile(cuis_style_with_freqs, 25))
    cuis_style_to_throw_away = list(data_cuis_style_top.Cuisine_Style.value_counts()\
                                    [data_cuis_style_top.Cuisine_Style.value_counts() < top_cuis_style_count].index)
    data_cuis_style_top.loc[data_cuis_style_top['Cuisine_Style'].isin(cuis_style_to_throw_away), 'Cuisine_Style'] = 'other_cuis_style'
    # Продолжим преобразование
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'VeganOptions'", "'Healthy'", "'GlutenFreeOptions'"], 
                                                                                     "'VegetarianFriendly'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Italian'", "'Spanish'", "'Portuguese'", "'Greek'", 
                                                                                      "'Turkish'", "'Moroccan'"], "'Mediterranean'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Pizza'", "'StreetFood'"], "'FastFood'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Japanese'", "'Sushi'", "'Chinese'", "'Thai'", 
                                                                                      "'Vietnamese'", "'Korean'", "'Indonesian'", 
                                                                                      "'Malaysian'", "'Tibetan'", "'Taiwanese'"], "'Asian'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Pub'", "'WineBar'", "'Gastropub'", "'BrewPub'"], "'Bar'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace("'Steakhouse'", "'Cafe'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Austrian'", "'Hungarian'", "'German'", "'Polish'", 
                                                                                      "'Slovenian'", "'Swiss'", "'Czech'", "'Belgian'", 
                                                                                      "'Croatian'"], "'CentralEuropean'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Halal'", "'Lebanese'", "'Israeli'", "'Persian'", 
                                                                                      "'Arabic'", "'Kosher'"], "'MiddleEastern'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Barbecue'", "'Grill'"], "'American'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Latin'", "'Argentinean'", "'Brazilian'", 
                                                                                      "'Peruvian'", "'Venezuelan'"], "'SouthAmerican'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Danish'", "'Swedish'", "'Norwegian'"], "'Scandinavian'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Caribbean'", "'Jamaican'", "'Cuban'"], "'CentralAmerican'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Delicatessen'", "'French'"], "'European'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Pakistani'", "'Nepali'", "'Bangladeshi'", 
                                                                                      "'SriLankan'"], "'Indian'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Ethiopian'", "'Tunisian'"], "'African'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace("'Contemporary'", "'Fusion'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Soups'", "'Balti'", "'Russian'", "'Afghani'"], 
                                                                                     "'EasternEuropean'")
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'British'", "'Dutch'", "'Irish'", "'Scottish'"], 
                                                                                     "'NorthWesternEuropean'")
    # Заполним оставшиеся пропуски значением, встречающимся чаще остальных.
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top['Cuisine_Style'].fillna(data_cuis_style_top['Cuisine_Style'].describe().top)
    # Продолжим сокращать количество уникальных значений
    cuis_style_with_freqs_sec = list(data_cuis_style_top.Cuisine_Style.value_counts())
    top_cuis_style_count_sec = int(np.percentile(cuis_style_with_freqs_sec, 10))
    cuis_style_to_throw_away_sec = list(data_cuis_style_top.Cuisine_Style.\
                                        value_counts()[data_cuis_style_top.Cuisine_Style.value_counts() < top_cuis_style_count_sec].index)
    data_cuis_style_top.loc[data_cuis_style_top['Cuisine_Style'].isin(cuis_style_to_throw_away_sec), 'Cuisine_Style'] = 'other_cuisine_style'
    # Преобразуем далее колонку
    data_cuis_style_top['Cuisine_Style'] = data_cuis_style_top.Cuisine_Style.replace(["'Diner'", "other_cuis_style", "'CentralAmerican'"], 
                                                                                     "other_cuisine_style") 
    # Перед созданием дамми-признаков, создадим копию столбца Cuisine_Style
    data_cuis_style_top['cuisine_style'] = data_cuis_style_top['Cuisine_Style'].copy()
    # Используем далее для заполнения пропусков в датафрейме df_output столбца Cuisine_Style
    top_cuis = data_cuis_style_top['cuisine_style'].describe().top
    data_cuis_style_top = pd.get_dummies(data_cuis_style_top, prefix = 'Cuis_Style', prefix_sep = '_',\
                                         columns = ['Cuisine_Style'], dummy_na=True)
    # Попробуем создать новый признак, создадим новый датафрейм data_cuis_style_new. Проведем его преобразование.
    data_cuis_style_new = pd.DataFrame(data_cuis_style_top.groupby(['index']).sum())
    # В датафрейме 50000 строк. Заменим все числа больше 0 на 1, а равные 0 на 0 (После группировки датафрейм data_cuis_style_top по index, 
    # мы получили общее количество дубликатов по каждой дамми-переменной признака 'тип кухни'. Там, где индекс был разбит на две строки 
    # со значением дамми-переменной равной 1, при группировке по индексу дамми-переменная стала равна 2, и т.д.). 
    # Преобразуем датафрейм.
    data_cuis_style_new['index'] = [i for i in range(len(data_cuis_style_new))]
    data_cuis_style_new = data_cuis_style_new[['index', "Cuis_Style_'African'", "Cuis_Style_'American'", "Cuis_Style_'Asian'", "Cuis_Style_'Bar'", 
                                               "Cuis_Style_'Cafe'", "Cuis_Style_'CentralEuropean'", "Cuis_Style_'EasternEuropean'", 
                                               "Cuis_Style_'European'", "Cuis_Style_'FastFood'", "Cuis_Style_'Fusion'", "Cuis_Style_'Indian'", 
                                               "Cuis_Style_'International'", "Cuis_Style_'Mediterranean'", "Cuis_Style_'Mexican'", 
                                               "Cuis_Style_'MiddleEastern'", "Cuis_Style_'NorthWesternEuropean'", "Cuis_Style_'Scandinavian'", 
                                               "Cuis_Style_'Seafood'", "Cuis_Style_'SouthAmerican'", "Cuis_Style_'VegetarianFriendly'", 
                                               'Cuis_Style_other_cuisine_style', 'Ranking']]
    def dummies(x):
        if x >= 1:
            return 1
        else:
            return 0
    for x in ["Cuis_Style_'African'", "Cuis_Style_'American'", "Cuis_Style_'Asian'", "Cuis_Style_'Bar'", "Cuis_Style_'Cafe'", 
              "Cuis_Style_'CentralEuropean'", "Cuis_Style_'EasternEuropean'", "Cuis_Style_'European'", "Cuis_Style_'FastFood'", 
              "Cuis_Style_'Fusion'", "Cuis_Style_'Indian'", "Cuis_Style_'International'", "Cuis_Style_'Mediterranean'", 
              "Cuis_Style_'Mexican'", "Cuis_Style_'MiddleEastern'", "Cuis_Style_'NorthWesternEuropean'", "Cuis_Style_'Scandinavian'", 
              "Cuis_Style_'Seafood'", "Cuis_Style_'SouthAmerican'", "Cuis_Style_'VegetarianFriendly'", 'Cuis_Style_other_cuisine_style']:
        data_cuis_style_new[x] = data_cuis_style_new[x].apply(lambda x: dummies(x))
    data_cuis_style_new = data_cuis_style_new.drop(['index', 'Ranking'], axis = 1)
    df_output = df_output.merge(data_cuis_style_new, how='inner', on='index')
    # Мы определили, что чаще всех встречается кухня 'VegetarianFriendly'. заполним пропуски в датафрейме df_output.
    df_output['Cuisine_Style'] = df_output['Cuisine_Style'].fillna(top_cuis)
    # Перед созданием дамми-признаков, создадим копию столбца city
    df_output['city'] = df_output['City'].copy()
    # Создадим dummy-признаки для нескольких переменных:
    df_output = pd.get_dummies(df_output, prefix = 'City', prefix_sep = '_', columns = ['City'], dummy_na=True)
    df_output = pd.get_dummies(df_output, prefix = 'Price_Range', prefix_sep = '_', columns = ['Price Range'], dummy_na=True)
        
    # ################### 4. Feature Engineering ####################################################
    # Создадим новый признак 'ranking_range'. Применим функцию, группирующую значения столбца 'ranking' (от 1 до 5)
    df_output['Ranking_range'] = df_output.Ranking.apply(rank_range)
    df_output['Ranking_range'] = np.sqrt(df_output.Ranking_range)**(1/30)
    df_output['c_London'] = df_output['Ranking'][df_output['city'] =='London']
    df_output['c_Paris'] = df_output['Ranking'][df_output['city'] =='Paris']
    df_output['c_Rome'] = df_output['Ranking'][df_output['city'] =='Rome']
    df_output['c_Barcelona'] = df_output['Ranking'][df_output['city'] =='Barcelona']
    df_output['c_Milan'] = df_output['Ranking'][df_output['city'] =='Milan']
    df_output['c_Madrid'] = df_output['Ranking'][df_output['city'] =='Madrid']
    df_output['c_other_city'] = df_output['Ranking'][df_output['city'] =='other_city']
    df_output['c_Berlin'] = df_output['Ranking'][df_output['city'] =='Berlin']
    df_output['c_Prague'] = df_output['Ranking'][df_output['city'] =='Prague']
    df_output['c_Lisbon'] = df_output['Ranking'][df_output['city'] =='Lisbon']
    df_output['c_Amsterdam'] = df_output['Ranking'][df_output['city'] =='Amsterdam']
    df_output['c_Vienna'] = df_output['Ranking'][df_output['city'] =='Vienna']
    df_output['c_Stockholm'] = df_output['Ranking'][df_output['city'] =='Stockholm']
    df_output['c_Zurich'] = df_output['Ranking'][df_output['city'] =='Zurich']
    df_output['c_Edinburgh'] = df_output['Ranking'][df_output['city'] =='Edinburgh']
    df_output['c_Copenhagen'] = df_output['Ranking'][df_output['city'] =='Copenhagen']
    df_output['c_Brussels'] = df_output['Ranking'][df_output['city'] =='Brussels']
    df_output['c_London'] = df_output['c_London'].fillna(df_output.c_London.mean())
    df_output['c_Paris'] = df_output['c_Paris'].fillna(df_output.c_Paris.mean())
    df_output['c_Rome'] = df_output['c_Rome'].fillna(df_output.c_Rome.mean())
    df_output['c_Barcelona'] = df_output['c_Barcelona'].fillna(df_output.c_Barcelona.mean())
    df_output['c_Milan'] = df_output['c_Milan'].fillna(df_output.c_Milan.mean())
    df_output['c_Madrid'] = df_output['c_Madrid'].fillna(df_output.c_Madrid.mean())
    df_output['c_other_city'] = df_output['c_other_city'].fillna(df_output.c_other_city.mean())
    df_output['c_Berlin'] = df_output['c_Berlin'].fillna(df_output.c_Berlin.mean())
    df_output['c_Prague'] = df_output['c_Prague'].fillna(df_output.c_Prague.mean())
    df_output['c_Lisbon'] = df_output['c_Lisbon'].fillna(df_output.c_Lisbon.mean())
    df_output['c_Amsterdam'] = df_output['c_Amsterdam'].fillna(df_output.c_Amsterdam.mean())
    df_output['c_Vienna'] = df_output['c_Vienna'].fillna(df_output.c_Vienna.mean())
    df_output['c_Stockholm'] = df_output['c_Stockholm'].fillna(df_output.c_Stockholm.mean())
    df_output['c_Zurich'] = df_output['c_Zurich'].fillna(df_output.c_Zurich.mean())
    df_output['c_Edinburgh'] = df_output['c_Edinburgh'].fillna(df_output.c_Edinburgh.mean())
    df_output['c_Copenhagen'] = df_output['c_Copenhagen'].fillna(df_output.c_Copenhagen.mean())
    df_output['c_Brussels'] = df_output['c_Brussels'].fillna(df_output.c_Brussels.mean())
    # Создадим новый признак 'numb_rev_range'. Применим функцию, группирующую значения столбца 'numb_of_reviews' (от 1 до 5)
    df_output['numb_rev_range'] = df_output['Number of Reviews'].apply(numb_review_range)
    df_output['numb_rev_1/30'] = np.sqrt(df_output.numb_rev_range[df_output.numb_rev_range > 0])**(1/30)
    df_output['numb_rev_range'] = np.sqrt(df_output.numb_rev_range[df_output.numb_rev_range > 0])**1.7
    # Создадим новые признаки для переменной 'ID_TA'
    df_output['id_ta_new'] = df_output['ID_TA']
    for k in range(df_output.ID_TA.value_counts()[0]+1):
        ID_TA_k = list(df_output.ID_TA.value_counts()[df_output.ID_TA.value_counts() == k].index)
        df_output.loc[df_output['id_ta_new'].isin(ID_TA_k), 'id_ta_new'] = k
    # Создадим новые признаки для переменной 'Restaurant_id'
    df_output['restaurant_id_new'] = df_output['Restaurant_id']
    for i in range(df_output.Restaurant_id.value_counts()[0]+1):
        restaurant_id_i = list(df_output.Restaurant_id.value_counts()[df_output.Restaurant_id.value_counts() == i].index)
        df_output.loc[df_output['restaurant_id_new'].isin(restaurant_id_i), 'restaurant_id_new'] = i
    # Создадим dummy-признаки для новых переменных:
    df_output = pd.get_dummies(df_output, prefix = 'id_ta_new', prefix_sep = '_', columns = ['id_ta_new'], dummy_na=True)
    df_output = pd.get_dummies(df_output, prefix = 'restaurant_id_new', prefix_sep = '_', columns = ['restaurant_id_new'], dummy_na=True)
        
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    df_output = df_output.drop(['index'], axis=1)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
test_data.head()

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
len(predict_submission)

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)